<a href="https://colab.research.google.com/github/akalpit23/Interpretable-ML/blob/main/Assignment_04_Interpretable_ML_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #04



## Akalpit Dawkhar

### **Interpretable ML-2**

### Overview

# 1. Importing necessary libraries and the data

In [1]:
pip install imodels --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import lime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Please use this to connect your GitHub repository to your Google Colab notebook
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Interpretable-ML" # Change to your repo name
git_path = 'https://github.com/akalpit23/Interpretable-ML/.git' #Change to your path
!git clone "{git_path}"

# Install dependencies from requirements.txt file
#!pip install -r "{os.path.join(repo_name,'requirements.txt')}" #Add if using requirements.txt

# Change working directory to location of notebook
notebook_dir = 'Assignment_04_Interpretable_ML_2.ipynb'
path_to_notebook = os.path.join(repo_name,notebook_dir)
%cd "{path_to_notebook}"
%ls

rm: ./sample_data: No such file or directory
Cloning into 'Interpretable-ML'...
remote: Not Found
fatal: repository 'https://github.com/akalpit23/Interpretable-ML/.git/' not found
[Errno 2] No such file or directory: 'Interpretable-ML/Assignment_04_Interpretable_ML_2.ipynb'
/Users/akalpitdawkhar/Desktop/School/SEM 3/XAI/Interpretable-ML/Assignment-4
Assignment_04_Interpretable_ML_2.ipynb  creditcard.csv


In [5]:
# Load the dataset
credit_card_data = pd.read_csv('creditcard.csv')

# Print the shape of the dataset
print(credit_card_data.shape)

# Print the first few rows of the dataset
print(credit_card_data.head())

(284807, 31)
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V

# 2. 

# 3.

# 4.